In [3]:
#! /usr/bin/env python

'''
Trains 7D QuaLiKiz-NN with a single output (efiTG)
'''

from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, BatchNormalization
from keras.optimizers import RMSprop, adam, Adam
from keras.initializers import TruncatedNormal
from keras import regularizers
from keras import backend as K
import pandas
import numpy
import sys
import os

# Define new Metric: rmse = Root Mean Square Error
def rmse(y_true, y_pred):
    return K.sqrt(K.mean(K.square( y_true-y_pred )))

# Gets the current file name. Useful for procedurally generating output/log files.
file_name =  os.path.basename(sys.argv[0][:-3])

# Define neural network parameters
batch_size = 10
#num_classes = 1
epochs = 100

# Load Data (which is in HDF5 or .h5 format)
store = pandas.HDFStore("../unstable_training_gen2_7D_nions0_flat_filter7.h5")
target_df = store['efeETG_GB'].to_frame()  # This one is relatively easy to train
input_df = store['input']

In [4]:
# Puts inputs and outputs in the same pandas dataframe.
# Also only keeps overlapping entries.
joined_dataFrame = target_df.join(input_df)

In [32]:
# Normalize data by standard deviation and mean-centering the data
joined_dataFrame['efeETG_GB'] = joined_dataFrame['efeETG_GB'] / joined_dataFrame['efeETG_GB'].std()
joined_dataFrame['efeETG_GB'] = (joined_dataFrame['efeETG_GB'] - joined_dataFrame['efeETG_GB'].mean())/joined_dataFrame['efeETG_GB'].std()

joined_dataFrame['Ati'] = joined_dataFrame['Ati'] / joined_dataFrame['Ati'].std()
joined_dataFrame['Ati'] = (joined_dataFrame['Ati'] - joined_dataFrame['Ati'].mean())/joined_dataFrame['Ati'].std()

joined_dataFrame['Ate'] = joined_dataFrame['Ate'] / joined_dataFrame['Ate'].std()
joined_dataFrame['Ate'] = (joined_dataFrame['Ate'] - joined_dataFrame['Ate'].mean())/joined_dataFrame['Ate'].std()

joined_dataFrame['An'] = joined_dataFrame['An'] / joined_dataFrame['An'].std()
joined_dataFrame['An'] = (joined_dataFrame['An'] - joined_dataFrame['An'].mean())/joined_dataFrame['An'].std()

joined_dataFrame['qx'] = joined_dataFrame['qx'] / joined_dataFrame['qx'].std()
joined_dataFrame['qx'] = (joined_dataFrame['qx'] - joined_dataFrame['qx'].mean())/joined_dataFrame['qx'].std()

joined_dataFrame['smag'] = joined_dataFrame['smag'] / joined_dataFrame['smag'].std()
joined_dataFrame['smag'] = (joined_dataFrame['smag'] - joined_dataFrame['smag'].mean())/joined_dataFrame['smag'].std()

joined_dataFrame['x'] = joined_dataFrame['x'] / joined_dataFrame['x'].std()
joined_dataFrame['x'] = (joined_dataFrame['x'] - joined_dataFrame['x'].mean())/joined_dataFrame['x'].std()

joined_dataFrame['Ti_Te'] = joined_dataFrame['Ti_Te'] / joined_dataFrame['Ti_Te'].std()
joined_dataFrame['Ti_Te'] = (joined_dataFrame['Ti_Te'] - joined_dataFrame['Ti_Te'].mean())/joined_dataFrame['Ti_Te'].std()

In [27]:
joined_dataFrame.describe(include='all')

,efeETG_GB,Ati,Ate,An,qx,smag,x,Ti_Te
count,638880.000000,6.388800e+05,638880.000000,638880.000000,638880.000000,638880.000000,638880.000000,638880.000000
mean,26.038572,5.439559e+00,8.509738,2.075249,4.355203,0.464310,0.483960,1.375224
std,22.899494,2.943392e+00,3.025638,1.693567,4.230269,1.157654,0.286345,0.675178
min,0.015788,1.000000e-14,2.000000,-5.000000,0.660000,-1.000000,0.090000,0.250000
25%,7.663502,3.500000e+00,6.500000,1.000000,1.500000,0.100000,0.210000,0.750000
50%,19.125839,5.000000e+00,8.000000,2.000000,2.500000,0.400000,0.450000,1.330000
75%,38.814079,7.250000e+00,10.000000,3.000000,5.000000,1.000000,0.690000,1.660000
max,116.856499,1.400000e+01,14.000000,6.000000,15.000000,5.000000,0.990000,2.500000


In [33]:
joined_dataFrame.describe(include='all')

,efeETG_GB,Ati,Ate,An,qx,smag,x,Ti_Te
count,6.388800e+05,6.388800e+05,6.388800e+05,6.388800e+05,6.388800e+05,6.388800e+05,6.388800e+05,6.388800e+05
mean,3.498177e-16,-4.076670e-13,1.726995e-13,1.942713e-13,1.051146e-12,-4.408851e-13,1.827640e-12,-3.604733e-12
std,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00
min,-1.136272e+00,-1.848058e+00,-2.151526e+00,-4.177720e+00,-8.735150e-01,-1.264894e+00,-1.375825e+00,-1.666559e+00
25%,-8.023415e-01,-6.589535e-01,-6.642361e-01,-6.349019e-01,-6.749461e-01,-3.146967e-01,-9.567490e-01,-9.260135e-01
50%,-3.018490e-01,-1.493374e-01,-1.684729e-01,-4.443218e-02,-4.385544e-01,-5.555202e-02,-1.185980e-01,-6.698017e-02
75%,5.578201e-01,6.150866e-01,4.925446e-01,5.460375e-01,1.524246e-01,4.627374e-01,7.195530e-01,4.217800e-01
max,3.965472e+00,2.908359e+00,1.814580e+00,2.317447e+00,2.516341e+00,3.918000e+00,1.767242e+00,1.665897e+00
